<a href="https://colab.research.google.com/github/Haseebasif7/Resnet-18/blob/main/Resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resnet 18

In [1]:
import torch
import torch.nn as nn
import torch.functional as F

In [2]:
class ResnetBlock(nn.Module):
    def __init__(self, in_channels, out_channels,down=False):
        super().__init__()

        stride = 2 if down else 1

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Downsample only if shape mismatch (when stride=2 or in_channels != out_channels)
        self.downsample = None
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity # Skip Connection
        out = F.relu(out)
        return out


In [3]:
class IniLayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=64,
            kernel_size=7,
            stride=2,
            padding=3,
            bias=False
        )
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    def forward(self, x):
        x = self.conv1(x)   # (B, 64, H, W) → (B, 64, H/2, W/2) -> due to stride 2
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool(x)    # (B, 64, H/2, W/2) → (B, 64, H/4, W/4)
        return x


In [4]:
import torch.nn as nn
import torch.nn.functional as F

class ResNet(nn.Module):
    def __init__(self, num_classes=1000):
        super().__init__()
        self.inilayer = IniLayer()

        # Each layer group contains 2 blocks
        self.layer1 = nn.Sequential(
            ResnetBlock(64, 64),
            ResnetBlock(64, 64)
        )

        self.layer2 = nn.Sequential(
            ResnetBlock(64, 128, down=True),  # stride=2 and projection
            ResnetBlock(128, 128)
        )

        self.layer3 = nn.Sequential(
            ResnetBlock(128, 256, down=True),
            ResnetBlock(256, 256)
        )

        self.layer4 = nn.Sequential(
            ResnetBlock(256, 512, down=True),
            ResnetBlock(512, 512)
        )

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # (B, 512, 1, 1)
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.inilayer(x)      # (B, 64, 56, 56)
        x = self.layer1(x)        # (B, 64, 56, 56)
        x = self.layer2(x)        # (B, 128, 28, 28)
        x = self.layer3(x)        # (B, 256, 14, 14)
        x = self.layer4(x)        # (B, 512, 7, 7)
        x = self.avgpool(x)       # (B, 512, 1, 1)
        x = torch.flatten(x, 1)   # (B, 512)
        x = self.fc(x)            # (B, num_classes)
        return x


In [5]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

# 1. Set Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),   #input size
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])

# 3. Load Dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset  = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)


100%|██████████| 170M/170M [00:03<00:00, 43.2MB/s]


In [6]:
model = ResNet(num_classes=10).to(device)

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [8]:
def train(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in tqdm(loader):
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    acc = 100 * correct / total
    return running_loss / len(loader), acc


In [9]:
def validate(model, loader, criterion):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

    acc = 100 * correct / total
    return val_loss / len(loader), acc


In [10]:
epochs = 100
for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc = validate(model, test_loader, criterion)

    print(f"[Epoch {epoch+1}] Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")


100%|██████████| 782/782 [02:41<00:00,  4.84it/s]


[Epoch 1] Train Loss: 1.4086, Train Acc: 48.70% | Val Loss: 1.0605, Val Acc: 62.51%


100%|██████████| 782/782 [02:38<00:00,  4.94it/s]


[Epoch 2] Train Loss: 0.8917, Train Acc: 68.39% | Val Loss: 0.9331, Val Acc: 68.71%


100%|██████████| 782/782 [02:38<00:00,  4.93it/s]


...

[Epoch 7] Train Loss: 0.2650, Train Acc: 90.70% | Val Loss: 0.6958, Val Acc: 78.63%


100%|██████████| 782/782 [02:38<00:00,  4.93it/s]


[Epoch 8] Train Loss: 0.1941, Train Acc: 93.13% | Val Loss: 0.5777, Val Acc: 83.21%


  4%|▍         | 31/782 [00:06<02:35,  4.83it/s]


KeyboardInterrupt: 

In [ ]:
# stopped at 8 epochs . Network is deep and its overfitting on Simple CIFAR dataset
# Was just for Explanation